In [13]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MXgprVLj_5_n0COvxsy_CZOsTpL1jwmk' -O Fastfood_Opinion.csv
%pip install pythainlp keras tensorflow

UsageError: Line magic function `%wget` not found.


In [11]:
import pandas as pd
from pythainlp import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import pandas as pd
import numpy as np
import re

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional, SimpleRNN
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

In [14]:
train = pd.read_csv("Fastfood_Opinion.csv")
x = train['message'].astype(str)
y = train['class']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

x_tts = tokenizer.texts_to_sequences(x)
maxlen = max([len(s) for s in x_tts])
x_pad = pad_sequences(x_tts, maxlen=maxlen, padding="post")

print("X_train_pad:\n", x_pad)

X_train, X_test, y_train, y_test = train_test_split(x_pad, y, test_size=0.2, random_state=42)

X_train_pad:
 [[  32   21    8 ...    0    0    0]
 [  45  353  354 ...    0    0    0]
 [   6    7    5 ...    0    0    0]
 ...
 [5268    0    0 ...    0    0    0]
 [5269   12  242 ...    0    0    0]
 [ 345    9    0 ...    0    0    0]]


## fully connected

In [15]:
vocab_size = len(tokenizer.word_index) + 1

# Fix Random Weight
# For reproducable
np.random.seed(7)

dl_model = Sequential()
dl_model.add(Embedding(input_dim=vocab_size,
                    output_dim=10,
                    input_length=maxlen))

dl_model.add(Flatten())

# relu: 0, z
dl_model.add(Dense(20, activation='relu')) 
dl_model.add(Dense(10, activation='relu'))

# sigmoid(1 node) & softmax(multi nodes): 0->1 prob
dl_model.add(Dense(1, activation='sigmoid')) 

# if softmax then binary_corssentropy -> crossentropy
dl_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

dl_model.summary()
dl_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 94, 10)            52700     
                                                                 
 flatten (Flatten)           (None, 940)               0         
                                                                 
 dense (Dense)               (None, 20)                18820     
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 71741 (280.24 KB)
Trainable params: 71741 (280.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
17/17 [

## lstm


In [16]:
vocab_size = len(tokenizer.word_index) + 1

# Fix Random Weight
# For reproducable
np.random.seed(7)

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size,
                    output_dim=10,
                    input_length=maxlen))

lstm_model.add(LSTM(64))

# relu: 0, z
lstm_model.add(Dense(3, activation='relu'))

# sigmoid(1 node) & softmax(multi nodes): 0->1 prob
lstm_model.add(Dense(1, activation='sigmoid'))

# if softmax then crossentropy
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.summary()
lstm_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 94, 10)            52700     
                                                                 
 lstm (LSTM)                 (None, 64)                19200     
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
 dense_4 (Dense)             (None, 1)                 4         
                                                                 
Total params: 72099 (281.64 KB)
Trainable params: 72099 (281.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
17/17 [==============================] - 2s 44ms/step - loss: 0.6920 - accuracy: 0.5556 - val_loss: 0.6873 - val_accuracy: 0.5736
Epoch 2

## rnn

In [17]:
vocab_size = len(tokenizer.word_index) + 1

# Fix Random Weight
# For reproducable
np.random.seed(7)

rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size,
                    output_dim=10,
                    input_length=maxlen))

rnn_model.add(SimpleRNN(64))

# relu: 0, z
rnn_model.add(Dense(3, activation='relu'))

# sigmoid(1 node) & softmax(multi nodes): 0->1 prob
rnn_model.add(Dense(1, activation='sigmoid'))

# if softmax then crossentropy
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn_model.summary()
rnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 94, 10)            52700     
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                4800      
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
 dense_6 (Dense)             (None, 1)                 4         
                                                                 
Total params: 57699 (225.39 KB)
Trainable params: 57699 (225.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
17/17 [==============================] - 1s 19ms/step - loss: 0.6922 - accuracy: 0.5166 - val_loss: 0.6995 - val_accuracy: 0.5426
Epoch 2

## bi-lstm

In [18]:
vocab_size = len(tokenizer.word_index) + 1

# Fix Random Weight
# For reproducable
np.random.seed(7)

bilstm_model = Sequential()
bilstm_model.add(Embedding(input_dim=vocab_size,
                    output_dim=10,
                    input_length=maxlen))

bilstm_model.add(Bidirectional(LSTM(64)))

# relu: 0, z
bilstm_model.add(Dense(3, activation='relu'))

# sigmoid(1 node) & softmax(multi nodes): 0->1 prob
bilstm_model.add(Dense(1, activation='sigmoid'))

# if softmax then crossentropy
bilstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

bilstm_model.summary()
bilstm_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 94, 10)            52700     
                                                                 
 bidirectional (Bidirection  (None, 128)               38400     
 al)                                                             
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 91491 (357.39 KB)
Trainable params: 91491 (357.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
17/17 [==============================] - 4s 61ms/step - loss: 0.6931 - 

# in conclusion

#### เปรียบเทียบผลลัพธ์ที่ได้
- lstm และ rnn มีแนวโน้มที่จะมีความแม่นยำที่สุดเมื่อเปรียบเทียบกับ accuracy และ lost ของตัว test